In [4]:
import os
import requests
from bs4 import BeautifulSoup
from gtts import gTTS
import moviepy as mp
import torch
from torchvision import transforms
from PIL import Image
import joblib

In [ ]:
def fetch_images(query, num_images=5):
    url = f"https://www.google.com/search?hl=en&tbm=isch&q={query}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to fetch images.")
        return []
    soup = BeautifulSoup(response.text, 'html.parser')
    return [img['src'] for img in soup.find_all('img')[1:num_images+1] if 'src' in img.attrs]

def download_image(image_url, folder_path="downloaded_images"):
    os.makedirs(folder_path, exist_ok=True)
    image_name = os.path.join(folder_path, image_url.split("/")[-1])
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        with open(image_name, 'wb') as file:
            file.write(response.content)
        return image_name
    return None

def create_video_from_image_and_text(image_path, text, output_filename="output_video.mp4"):
    tts = gTTS(text, lang='en')
    tts.save("temp_audio.mp3")
    clip = mp.ImageClip(image_path).set_duration(5).set_audio(mp.AudioFileClip("temp_audio.mp3"))
    clip.write_videofile(output_filename, fps=24)
    os.remove("temp_audio.mp3")

def extract_text_from_image(image_path):
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img_tensor)
    return "Extracted Text"  # Replace with decoding logic


In [ ]:
if _name_ == "_main_":
    image_path = "maggi_recipe.jpg"  # Example input
    text = extract_text_from_image(image_path)
    print(f"Extracted Text: {text}")
    images = fetch_images(text, 3)
    for url in images:
        downloaded_path = download_image(url)
        if downloaded_path:
            create_video_from_image_and_text(downloaded_path, text)